In [1]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from plotnine import *
import matplotlib.pyplot as plt
import pandas as pd
import glob

In [2]:
runs = [f for f in glob.glob("KMPlotter_result/*.txt")]
runs

['KMPlotter_result/KM_SS2_MULTI_E_RFS.txt',
 'KMPlotter_result/KM_SS2_MULTI_H_RFS.txt',
 'KMPlotter_result/KM_SS2_MULTI_M_RFS.txt']

In [3]:
p_value_df = pd.DataFrame()
for i in runs:
    data_name = i.split('.')[0]
    kmf = KaplanMeierFitter()
    kmf_df = pd.DataFrame()
    KM_df = pd.read_csv(i, sep='\t',index_col=0)
    KM_df['Expression (1=high)'] = KM_df['Expression (1=high)'].replace(1,'high')
    KM_df['Expression (1=high)'] = KM_df['Expression (1=high)'].replace(0,'low')
    for cat in ['low','high']:
        f = KM_df['Expression (1=high)']==cat
        T = KM_df[f]['Time (months)']
        C = KM_df[f]['Event']
        kmf.fit(T, event_observed=C, label=['RFS'])
        temp_df = kmf.survival_function_.reset_index()
        temp_df['label'] = cat
        kmf_df = kmf_df.append(temp_df)
        
    f = KM_df['Expression (1=high)']=='low'
    T = KM_df[f]['Time (months)']
    C = KM_df[f]['Event']

    f2 = KM_df['Expression (1=high)']=='high'
    T2 = KM_df[f2]['Time (months)']
    C2 = KM_df[f2]['Event']


    results = logrank_test(T, T2, C, C2, alpha=.95)
    
    p_value_df[data_name] = [results.p_value]
    
    
    plot1 = (ggplot(kmf_df)
        + theme_bw()
        + geom_step(aes('timeline','RFS',group='label',color='label'))+ ylim(0, 1)
         +scale_color_manual({'low':'black', 'high':'red'})
         +theme(
          strip_background=element_blank(), strip_text_x=element_blank(),
          plot_background=element_rect(fill='white', alpha=.3),panel_background=element_rect(fill='white', alpha=.2),figure_size=(5, 5),
          panel_border = element_rect(colour = "black", fill=None, size=1),panel_grid_major = element_blank(),panel_grid_minor = element_blank()
          ))
    plot1
        
    save_as_pdf_pages([plot1],data_name+'_ref.pdf')
        
    plot2 = (ggplot(kmf_df)
        + theme_bw()
        + geom_step(aes('timeline','RFS',group='label',color='label'),show_legend = False,size=1.8)+ ylim(0, 1)
         +scale_color_manual({'low':'black', 'high':'red'})
         +theme(axis_text_x=element_blank(),axis_text_y=element_blank(),axis_title_x=element_blank(),axis_title_y=element_blank(),
          strip_background=element_blank(), strip_text_x=element_blank(),
          plot_background=element_rect(fill='white', alpha=.3),panel_background=element_rect(fill='white', alpha=.2),figure_size=(5, 5),
          panel_border = element_rect(colour = "black", fill=None, size=1),panel_grid_major = element_blank(),panel_grid_minor = element_blank()
          ))
    save_as_pdf_pages([plot2],data_name+'.pdf')
    
    print(len(KM_df))

    print('-'*100)

/home/weilun/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:820: PlotnineWarning: Filename: KMPlotter_result/KM_SS2_MULTI_E_RFS_ref.pdf


2032
----------------------------------------------------------------------------------------------------


/home/weilun/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:820: PlotnineWarning: Filename: KMPlotter_result/KM_SS2_MULTI_E_RFS.pdf
/home/weilun/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:820: PlotnineWarning: Filename: KMPlotter_result/KM_SS2_MULTI_H_RFS_ref.pdf


4929
----------------------------------------------------------------------------------------------------


/home/weilun/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:820: PlotnineWarning: Filename: KMPlotter_result/KM_SS2_MULTI_H_RFS.pdf
/home/weilun/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:820: PlotnineWarning: Filename: KMPlotter_result/KM_SS2_MULTI_M_RFS_ref.pdf
/home/weilun/anaconda3/lib/python3.8/site-packages/plotnine/ggplot.py:820: PlotnineWarning: Filename: KMPlotter_result/KM_SS2_MULTI_M_RFS.pdf


2032
----------------------------------------------------------------------------------------------------


In [4]:
p_value_df.to_csv('p_values.csv')

In [5]:
data_name

'KMPlotter_result/KM_SS2_MULTI_M_RFS'